<a href="https://colab.research.google.com/github/shailymishra/Paper-Presentation-Summary-Implementation/blob/main/MD_in_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Imports
import torch

import os
import numpy as np
import pandas as pd
import pandas.util.testing as tm
from tqdm import tqdm
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from torch import nn, optim

import torch.nn.functional as F


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import sys


In [ ]:
class MDusingNN(nn.Module):
    def __init__(self, n_agents, k):
        super(MDusingNN, self).__init__()
        self.m = n_agents
        self.k = k
        self.fc = nn.ModuleList([ nn.Linear(1, self.k-1 )   for i in range(self.m)])
        self.prices = nn.Linear(1,self.k-1)
        torch.nn.init.uniform_(self.prices.weight, a=0.0, b=1.0)
        self.prices.bias.data.fill_(0.0)

        self.init_buyers_network()

    def init_buyers_network(self):
        self.n_discretevalues = [100,100]
        self.discretevalues = []
        for n in self.n_discretevalues:
          # values = np.random.rand(n)
          values = np.random.uniform(0,1,n)
          self.discretevalues.append(values)
        
        self.valuation = []
        ## for first item
        value = torch.ones((self.n_discretevalues), dtype=torch.float64)
        for i in range(self.n_discretevalues[0]):
          value[i] = self.discretevalues[0][i]  
        self.valuation.append(value)

        ## for second item
        value = torch.ones((self.n_discretevalues), dtype=torch.float64)
        for i in range(self.n_discretevalues[1]):
          value[:,i] = self.discretevalues[1][i]  
        self.valuation.append(value)

    
    def forward(self):
      zerotensor = torch.tensor([0.0])
      constantinput = torch.tensor([1.0])
      allocationarr = []
      prices = torch.relu(self.prices(constantinput))
      prices = torch.cat([prices,zerotensor])
      for row in range(self.m):
        output = torch.sigmoid(self.fc[row](constantinput))
        output = torch.cat([output,zerotensor])
        allocationarr.append(output)
      allocationarr = torch.stack(allocationarr) 
      return allocationarr, prices

    def buyer_network(self, allocationarr,prices):
      productvalue = np.prod(self.n_discretevalues) 
      newdim = self.n_discretevalues.copy()
      newdim.append(self.k)
      valuationForAllocation = []
      valuation_item1 = torch.reshape(torch.mm( torch.reshape( self.valuation[0], (productvalue,1)).double()   ,  
                                              torch.reshape( allocationarr[0], (1,menu_size)).double() ), newdim)
      
      valuation_item2 = torch.reshape(torch.mm( torch.reshape( self.valuation[1], (productvalue,1)).double()   ,  
                                              torch.reshape( allocationarr[1], (1,menu_size)).double() ), newdim)
      valuationForAllocation.append(valuation_item1)
      valuationForAllocation.append(valuation_item2)
      totalvaluationforallocation = torch.zeros((newdim))
      for valuation in valuationForAllocation :
        totalvaluationforallocation = totalvaluationforallocation + valuation
      
      utility = totalvaluationforallocation - prices

      m = nn.Softmax(dim=2)  ## dim for softmax
      utility = m(utility)

      return utility, prices


    def loss_function(self, utility,prices):
      print('Loss function')
      dummyvar = torch.zeros((utility.shape))
      dummyvar += prices
      maxu,indices = torch.max(utility, dim=2)
      revenue = torch.mean(prices[indices])
      print('revenue', revenue)
      loss = -revenue
      return loss


In [ ]:
n_items = 2
menu_size = 4
net = MDusingNN(n_items,menu_size)

def train(epochs=25, learning_rate=0.1):
  optimizer = optim.Adam(net.parameters(), lr= learning_rate)
  
  with torch.autograd.set_detect_anomaly(True):
    for epoch in range(epochs):
        optimizer.zero_grad()
        allocationarr, prices = net()
        print('allocationarr', allocationarr)
        print('shape', allocationarr.shape )
        print('prices  ', prices)
        utility,prices= net.buyer_network(allocationarr,prices)
        train_loss = net.loss_function(utility,prices)
        print('loss...', train_loss)
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

train()

## sigmoid doubt
## how many discretization to do

## network is really smart - i was trying to maximizing utility so the network gave 0 prices
## i need to maximize revenue


allocationarr tensor([[0.4512, 0.2849, 0.6532, 0.0000],
        [0.4017, 0.1331, 0.2078, 0.0000]], grad_fn=<StackBackward>)
shape torch.Size([2, 4])
prices   tensor([0.9386, 0.4639, 0.9323, 0.0000], grad_fn=<CatBackward>)
Loss function
revenue tensor(0., grad_fn=<MeanBackward0>)
loss... tensor(-0., grad_fn=<NegBackward>)
allocationarr tensor([[0.4512, 0.2849, 0.6532, 0.0000],
        [0.4017, 0.1331, 0.2078, 0.0000]], grad_fn=<StackBackward>)
shape torch.Size([2, 4])
prices   tensor([0.9386, 0.4639, 0.9323, 0.0000], grad_fn=<CatBackward>)
Loss function
revenue tensor(0., grad_fn=<MeanBackward0>)
loss... tensor(-0., grad_fn=<NegBackward>)
allocationarr tensor([[0.4512, 0.2849, 0.6532, 0.0000],
        [0.4017, 0.1331, 0.2078, 0.0000]], grad_fn=<StackBackward>)
shape torch.Size([2, 4])
prices   tensor([0.9386, 0.4639, 0.9323, 0.0000], grad_fn=<CatBackward>)
Loss function
revenue tensor(0., grad_fn=<MeanBackward0>)
loss... tensor(-0., grad_fn=<NegBackward>)
allocationarr tensor([[0.4512, 

In [ ]:

## For two items, additive valuation, 
## between 0 to 1
## uniform so 1/n 
# def buyersNetwork(allocationarr,prices):
#   ## d1xd2xd_n_aitems
#   n_discretevalues = [2,3]
#   discretevalues = []
#   for n in n_discretevalues:
#     # values = np.random.rand(n)
#     values = np.random.uniform(0,1,n)

#     discretevalues.append(values)
#   # print('_____________________________________')
#   # print('discrete values')
#   # print(discretevalues)
#   # print('_____________________________________')

#   values = []
#   ## for first item
#   value = torch.ones((n_discretevalues), dtype=torch.float64)
#   for i in range(n_discretevalues[0]):
#     value[i] = discretevalues[0][i]  
#   # print('value', value)
#   values.append(value)

#   ## for second item
#   value = torch.ones((n_discretevalues), dtype=torch.float64)
#   for i in range(n_discretevalues[1]):
#     value[:,i] = discretevalues[1][i]  
#   # print('value', value)
#   values.append(value)

#   # print('_______________________________________________')
#   # print('   Alloation Array     ')
#   # print(allocationarr)
#   # print('_______________________________________________')
#   # print('   Prices     ')
#   # print(prices)
#   # print('_______________________________________________')
#   # res1 = torch.reshape(torch.mm( torch.reshape( v0, (24,1))   ,   torch.reshape( x0, (1,5)) ), (2,3,4,5))

#   # print('V0 is', values[0], 'shape is ',values[0].shape )
#   # print('X0 is ',allocationarr[0] ,'  shape is ',allocationarr[0].shape)
#   productvalue = np.prod(n_discretevalues) 
#   newdim = n_discretevalues.copy()
#   newdim.append(menu_size)
#   valuationForAllocation = []
#   valuation_item1 = torch.reshape(torch.mm( torch.reshape( values[0], (productvalue,1)).double()   ,  
#                                            torch.reshape( allocationarr[0], (1,menu_size)).double() ), newdim)
  
#   valuation_item2 = torch.reshape(torch.mm( torch.reshape( values[1], (productvalue,1)).double()   ,  
#                                            torch.reshape( allocationarr[1], (1,menu_size)).double() ), newdim)

#   print('______________________________________________________________')
#   print('  valuation_item1   ')
#   print(valuation_item1)
#   print('______________________________________________________________')

#   print('______________________________________________________________')
#   print('  valuation_item2   ')
#   print(valuation_item2)
#   print('______________________________________________________________')


#   valuationForAllocation.append(valuation_item1)
#   valuationForAllocation.append(valuation_item2)

#   totalvaluationforallocation = torch.zeros((newdim))
#   for valuation in valuationForAllocation :
#     totalvaluationforallocation = totalvaluationforallocation + valuation
  
#   print('______________________________________________________________')
#   print('  Total Valuation for allocation   ')
#   print(totalvaluationforallocation)
#   print('______________________________________________________________')

#   print('______________________________________________________________')
#   print(' price ')
#   print(prices)
#   print('______________________________________________________________')

#   utility = totalvaluationforallocation - prices
#   print('______________________________________________________________')
#   print(' utility ')
#   print(utility)
#   print('______________________________________________________________')
#   # m = nn.Softmax(dim=2)  ## dim for softmax
#   # utility = m(utility)
#   # print('______________________________________________________________')
#   # print(' utility ')
#   # print(utility)
#   # print('______________________________________________________________')

#   maxu,indices = torch.max(utility, dim=2)
#   print('maxu...', maxu)

#   loss = torch.mean(maxu)
#   print('loss..', loss)

